In [ ]:
import json
import pandas as pd
import MeCab
import matplotlib.pyplot as plt
import csv
import numpy as np
from wordcloud import WordCloud
from janome.analyzer import Analyzer
from janome.charfilter import UnicodeNormalizeCharFilter, RegexReplaceCharFilter
from janome.tokenizer import Tokenizer as JanomeTokenizer  # sumyのTokenizerと名前が被るため
from janome.tokenfilter import POSKeepFilter, ExtractAttributeFilter
import re
import ipadic
from tqdm import tqdm
import unicodedata
#Algos
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.reduction import ReductionSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.kl import KLSummarizer
from janome.tokenizer import Tokenizer as JanomeTokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
from sumy.utils import get_stop_words

import pkg_resources, imp
imp.reload(pkg_resources)
import unicodedata
import spacy
nlp = spacy.load('ja_ginza')#spacyの日本語モデル(Ginza)をロード

from ginza import *
import neologdn
import re
import emoji
import mojimoji
from collections import Counter



In [ ]:
import sys
import pandas as pd
#
import sqlite3
# ------------------------------------------------------------------
sys.stderr.write("*** 開始 ***\n")
#読み込み先
file_company = "./datas/OpenWork_company.db"
file_company_info = './datas/OpenWork_company_info.db'

conn = sqlite3.connect(file_company)
conn2 = sqlite3.connect(file_company_info)
#
company_df=pd.read_sql_query('SELECT * FROM df_company', conn)
company_info_df=pd.read_sql_query('SELECT * FROM df_company_info', conn2)
display(company_df.head(15))
display(company_info_df.head(15))
#
conn.close()
conn2.close()
sys.stderr.write("*** 終了 ***\n")

#複数行に値が重複している可能性があるためdrop_duplicatesで、1行1ユニーク値にする
company_df = company_df.drop_duplicates()
company_info_df = company_info_df.drop_duplicates()

company_info_df.tail()

In [ ]:
import numpy as np
import pandas as pd

#company_dfの前処理
company_df.本文 = company_df.本文.str.replace('\u3000', ' ')
company_df.本文 = company_df.本文.str.replace('■', ' ')
company_df.本文 = company_df.本文.str.replace('⇒', ' ')
company_df.本文 = company_df.本文.str.replace('→', ' ')
company_df.本文 = company_df.本文.str.replace('□', ' ')
company_df.本文 = company_df.本文.str.replace('：', ' ')
company_df.本文 = company_df.本文.str.replace('▷', ' ')
#company_info_dfの前処理
def preprocessing(company_info_df):
        
    #まず、'--'が含まれている場合はNaNに置き換える
    replace_line = company_info_df.replace('--',None)

    #欠損していない値が3未満の企業はdrop
    processed_company_info_df = replace_line.dropna(axis=1, thresh=3,  inplace=False)

    #obj→floatに変換
    processed_company_info_df.月残業時間 = processed_company_info_df.月残業時間.astype('float')
    processed_company_info_df.有給消化率 = processed_company_info_df.有給消化率.astype('float')
    processed_company_info_df.平均年収  = processed_company_info_df.平均年収.astype('float')
    processed_company_info_df.総合スコア = processed_company_info_df.総合スコア.astype('float')
    processed_company_info_df.社員の士気 = processed_company_info_df.社員の士気.astype('float')
    processed_company_info_df.風通し = processed_company_info_df.風通し.astype('float')
    processed_company_info_df.社員相互尊重 = processed_company_info_df.社員相互尊重.astype('float')
    processed_company_info_df['20代成長性'] = processed_company_info_df['20代成長性'].astype('float')
    processed_company_info_df.長期育成 = processed_company_info_df.長期育成.astype('float')
    processed_company_info_df.コンプラ = processed_company_info_df.コンプラ.astype('float')
    processed_company_info_df.評価納得感 = processed_company_info_df.評価納得感.astype('float')
    processed_company_info_df.待遇満足度 = processed_company_info_df.待遇満足度.astype('float')
    
    #Nullは中央値で埋める
    processed_company_info_df = processed_company_info_df.fillna(value=processed_company_info_df.median())
    
    #その他、分析に必要なdfの必要な前処理
    processed_company_info_df['会社年齢'] = 2022 - processed_company_info_df['設立年']
    processed_company_info_df.drop('設立年',axis=1,inplace=True)
    
    return processed_company_info_df

company_info_df_processed = preprocessing(company_info_df)

In [ ]:
#stopword list 
stopword_list = []
with open('./stopwords/stopwords.csv','r',encoding='utf-8') as f:
    text = f.readlines()
    for tk in text:
        tk = tk.strip('\n')
        tk = tk.replace('\ufeff','')
        stopword_list.append(tk)


In [ ]:
#指定した企業IDの企業コメントを可視化し、要約する関数

#企業ID指定
company_name = 'グーグル合同会社'

#algos

algorithm_dic = {"lex": LexRankSummarizer(), "tex": TextRankSummarizer(), "lsa": LsaSummarizer(),\
                 "kl": KLSummarizer(), "luhn": LuhnSummarizer(), "redu": ReductionSummarizer(),\
                 "sum": SumBasicSummarizer()}

#日本語処理用のクラス
class JapaneseCorpus:
    # ①
    def __init__(self):
        self.nlp = spacy.load('ja_ginza')
        self.analyzer = Analyzer(
            char_filters=[UnicodeNormalizeCharFilter(), RegexReplaceCharFilter(r'[(\)「」、。]', ' ')],  # ()「」、。は全てスペースに置き換える
            tokenizer=JanomeTokenizer(),
            token_filters=[POSKeepFilter(['名詞', '形容詞', '副詞', '動詞']), ExtractAttributeFilter('base_form')]  # 名詞・形容詞・副詞・動詞の原型のみ
        )

    # ②
    def preprocessing(self, text):#前処理の関数
        text = re.sub(r'\n', '', text)
        text = re.sub(r'\r', '', text)
        #text = re.sub(r'\d', '', text)
        text = re.sub(r'\s', '', text)
        text.replace('\u3000', '')
        text = text.lower()
        text = mojimoji.zen_to_han(text, kana=True)
        text = mojimoji.han_to_zen(text, digit=False, ascii=False)
        text = ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)
        text = neologdn.normalize(text)

        return text

    # ③
    def make_sentence_list(self, sentences):
        doc = self.nlp(sentences)
        self.ginza_sents_object = doc.sents #与えられた文字列を文章単位に分割
        sentence_list = [s for s in doc.sents]

        return sentence_list

    # ④
    def make_corpus(self):
        corpus = [' '.join(self.analyzer.analyze(str(s))) + '。' for s in self.ginza_sents_object]

        return corpus


#WordCloudで可視化と、レビューの要約双方を行うクラス
class company_summarize:
    def __init__(self,company_name):
        self.company_name = company_name
        self.company_id = company_df[company_df.企業名 == company_name].企業ID.unique()[0]#compane_nameをIDに変換
        self.nlp = spacy.load('ja_ginza')
        self.analyzer = Analyzer(
            char_filters=[UnicodeNormalizeCharFilter(), RegexReplaceCharFilter(r'[(\)「」、。]', ' ')],  # ()「」、。は全てスペースに置き換える
            tokenizer=JanomeTokenizer(),
            token_filters=[POSKeepFilter(['名詞', '形容詞', '副詞', '動詞']), ExtractAttributeFilter('base_form')]  # 名詞・形容詞・副詞・動詞の原型のみ
        )

    
    #Wordcloudでの可視化
    def visualize(self):
        CHASEN_ARGS = r' -F "%m\t%f[7]\t%f[6]\t%F-[0,1,2,3]\t%f[4]\t%f[5]\n"'
        CHASEN_ARGS += r' -U "%m\t%m\t%m\t%F-[0,1,2,3]\t\t\n"'
        tagger = MeCab.Tagger(ipadic.MECAB_ARGS + CHASEN_ARGS)
        #null parse
        tagger.parse('')
    
        #指定した企業IDのみに絞る
        company_text = pd.DataFrame(company_df[company_df.企業名 == self.company_name])

        #dfの本文から口コミデータを抽出し、strで連結しテキストデータへ
        all_text = company_text.本文.str.cat()
    
        #正規化
        all_text_norm = unicodedata.normalize("NFKC", all_text)
    
        #parse
        node = tagger.parseToNode(all_text_norm)

        # 指定した品詞を抽出しリストに
        word_list = []
    
        while node:
            word_type = node.feature.split(',')[0]
            if word_type in ["名詞", "動詞", "形容詞",'副詞']:
                word_list.append(node.surface)
            #ここのインデントミスると無限ループ
            node = node.next

        # リストを文字列に変換
        word_chain = ' '.join(word_list)
    
        #ストップワードリストをコピー
        stopword_list_indef = stopword_list
        
        #word_listの要素数の10%は、最も登場頻度が高いワードなのでストップリストに追加する
        fdist = Counter(word_list)
        Common_Words = fdist.most_common(n=int(len(fdist)*0.10))
        
        for common_word in Common_Words:
            stopword_list_indef.append(common_word[0])

        #社名が入らないように、stopwordlistに追加
        stopword_list_indef.append(company_text.企業名.drop_duplicates().str.cat().replace('株式会社',''))

        # ワードクラウド作成
        W = WordCloud(width=640, height=480, background_color='black', colormap='cool_r',
                      font_path='C:\Windows\Fonts\yumin.ttf', stopwords = set(stopword_list_indef)).generate(word_chain)
        # 表示設定
        plt.figure(figsize = (15, 12))
        plt.axis('off')

        plt.imshow(W)
        
    #前処理、コーパスリスト作成など
    def preprocessing(self, text):
        text = re.sub(r'\n', '', text)
        text = re.sub(r'\r', '', text)
        #text = re.sub(r'\d', '', text)
        text = re.sub(r'\s', '', text)
        text.replace('\u3000', '')
        text = text.lower()
        text = mojimoji.zen_to_han(text, kana=True)
        text = mojimoji.han_to_zen(text, digit=False, ascii=False)
        text = ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)
        text = neologdn.normalize(text)

        return text

    def make_sentence_list(self, sentences):
        doc = self.nlp(sentences)
        self.ginza_sents_object = doc.sents
        sentence_list = [s for s in doc.sents]

        return sentence_list

    def make_corpus(self):
        corpus = [' '.join(self.analyzer.analyze(str(s))) + '。' for s in self.ginza_sents_object]

        return corpus
    
    #文章内容要約
    def summarize_sentences(self,sentences_count=10, algorithm="lex", language="japanese"):
        
        company_split_df = pd.DataFrame(company_df[company_df.企業名 == self.company_name])
        if company_split_df.shape[0] >= 55:
            company_split_df = company_split_df.sample(n=55)
        #任意の企業名のレビューstrを取り出す
        all_text = company_split_df.本文.str.cat()
        all_text_norm = unicodedata.normalize("NFKC", all_text)
        sentences=' '.join(all_text_norm)
        
        #JPコーパスクラス継承
        corpus_maker = JapaneseCorpus()
        preprocessed_sentences = corpus_maker.preprocessing(sentences)
        preprocessed_sentence_list = corpus_maker.make_sentence_list(preprocessed_sentences)
        corpus = corpus_maker.make_corpus()
        parser = PlaintextParser.from_string(" ".join(corpus), Tokenizer(language))

        try:
            summarizer = algorithm_dic[algorithm]
        except KeyError:
            print("algorithm name:'{}'is not found.".format(algorithm))

        summarizer.stop_words = get_stop_words(language)
        #sentences_countは文書に一定の割合をかけた値でもよいが、読みやすさ重視で10センテンス固定長とする
        #int(len(corpus)*0.1)
        summary = summarizer(document=parser.document, sentences_count=10)
        
        
        if language == "japanese":
            return str("".join([str(preprocessed_sentence_list[corpus.index(sentence.__str__())]) for sentence in summary]))
        else:
            return str("".join([sentence.__str__() for sentence in summary]))

#run
if __name__ == '__main__':
    C = company_summarize(company_name)
    
    #結果の表示
    print('\033[1m' + C.company_name + '\033[0m'+ '\n')
    #company_info_dfから該当企業のスコアを表示
    print('\033[1m' + '該当企業の分野別評価' + '\033[0m'+ '\n')
    display(company_info_df_processed[company_info_df_processed.企業名 == company_name].drop('業種',axis=1))
    #該当企業の業種の平均スコアを表示
    print('\033[1m' + '該当企業の業種平均評価' + '\033[0m' + '\n')
    
    #業種平均を表示
    company_info_mean = company_info_df_processed.groupby('業種').agg(np.mean).round(2)
    display(company_info_mean[company_info_mean.index == company_info_df[company_info_df.企業名 == company_name].業種.str.cat()])

    print('------------------------')
    #WordCloudを表示する
    C.visualize()

    #文書要約を表示 読みやすさ重視で10センテンス固定長とする
    print(C.summarize_sentences(sentences_count=10, algorithm="luhn", language="japanese"))

In [ ]:
#thanks to https://zenn.dev/shikumiya_hata/articles/b18e362e2eae09
#内容要約
#前処理を集約したクラス
class JapaneseCorpus:
    # ①
    def __init__(self):
        self.nlp = spacy.load('ja_ginza')
        self.analyzer = Analyzer(
            char_filters=[UnicodeNormalizeCharFilter(), RegexReplaceCharFilter(r'[(\)「」、。]', ' ')],  # ()「」、。は全てスペースに置き換える
            tokenizer=JanomeTokenizer(),
            token_filters=[POSKeepFilter(['名詞', '形容詞', '副詞', '動詞']), ExtractAttributeFilter('base_form')]  # 名詞・形容詞・副詞・動詞の原型のみ
        )

    # ②
    def preprocessing(self, text):
        text = re.sub(r'\n', '', text)
        text = re.sub(r'\r', '', text)
        text = re.sub(r'\d', '', text)
        text = re.sub(r'\s', '', text)
        text.replace('\u3000', '')
        text = text.lower()
        text = mojimoji.zen_to_han(text, kana=True)
        text = mojimoji.han_to_zen(text, digit=False, ascii=False)
        text = ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)
        text = neologdn.normalize(text)

        return text

    # ③
    def make_sentence_list(self, sentences):
        doc = self.nlp(sentences)
        self.ginza_sents_object = doc.sents
        sentence_list = [s for s in doc.sents]

        return sentence_list

    # ④
    def make_corpus(self):
        corpus = [' '.join(self.analyzer.analyze(str(s))) + '。' for s in self.ginza_sents_object]

        return corpus

class EnglishCorpus(JapaneseCorpus):
    # ①
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')

    # ②
    def preprocessing(self, text):
        text = re.sub(r'\n', '', text)
        text = re.sub(r'\r', '', text)
        text = mojimoji.han_to_zen(text, digit=False, ascii=False)
        text = mojimoji.zen_to_han(text, kana=True)
        text = ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)
        text = neologdn.normalize(text)        

        return text

    # ④
    def make_corpus(self):
        corpus = []
        for s in self.ginza_sents_object:
            tokens = [str(t) for t in s]
            corpus.append(' '.join(tokens))

        return corpus


algorithm_dic = {"lex": LexRankSummarizer(), "tex": TextRankSummarizer(), "lsa": LsaSummarizer(),\
                 "kl": KLSummarizer(), "luhn": LuhnSummarizer(), "redu": ReductionSummarizer(),\
                 "sum": SumBasicSummarizer()}

#文章の内容を要約する関数
def summarize_sentences(sentences, sentences_count, algorithm="lex", language="japanese"):
    # ①
    if language == "japanese":
        corpus_maker = JapaneseCorpus()
    else:
        corpus_maker = EnglishCorpus()
    preprocessed_sentences = corpus_maker.preprocessing(sentences)
    preprocessed_sentence_list = corpus_maker.make_sentence_list(preprocessed_sentences)
    corpus = corpus_maker.make_corpus()
    parser = PlaintextParser.from_string(" ".join(corpus), Tokenizer(language))

    # ②
    try:
        summarizer = algorithm_dic[algorithm]
    except KeyError:
        print("algorithm name:'{}'is not found.".format(algorithm))

    summarizer.stop_words = get_stop_words(language)
    #sentences_countは文書に一定の割合をかけた値でもよいが、読みやすさ重視で10センテンス固定長とする
    #int(len(corpus)*0.1)
    summary = summarizer(document=parser.document, sentences_count=10)

    # ③
    if language == "japanese":
        return ("".join([str(preprocessed_sentence_list[corpus.index(sentence.__str__())]) for sentence in summary]))
    else:
        return (" ".join([sentence.__str__() for sentence in summary]))

In [ ]:
#summarize quality test

from sumeval.metrics.rouge import RougeCalculator

#read human summarization csv
sum_df_t = pd.read_csv('C:/Users/Administrator/Desktop/Python 3/Graduation Task/human_sum/ss50.csv',encoding='cp932')
sum_df_t.人間要約 = sum_df_t.人間要約.str.replace('\n','')

#specify ja
nlp = spacy.load('ja_ginza')
rouge = RougeCalculator(stopwords=True, lang="ja")


In [ ]:
#measuring score on each 20 summaries
score_dict = {}
score_list = []

for i in range(20):
    print('{}:処理中'.format(i))
    for algo in algorithm_dic.keys():
        #print(i,algo)
        summary = str((summarize_sentences(sentences=sum_df_t.本文[i], sentences_count=10, algorithm=algo, language="japanese")))
        rouge_2 = rouge.rouge_n(summary=summary,references=str(sum_df_t.人間要約[i]),n=2)
        score_dict[algo] = rouge_2
    score_list.append(score_dict)
    #dict 初期化
    score_dict = {}


In [ ]:
result = pd.DataFrame(score_list)
result.describe()

In [ ]:
#spacyアナライザ
nlp = spacy.load('ja_ginza')
ginza.set_split_mode(nlp, "C") # 形態素の分割モード指定
def spacy_analyzer(text):
    #正規化
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r'\u3000','',text)
    text.replace('\u3000', '')
    doc = nlp(text)

    #結果を格納するリスト
    result_list = []
    for sent in doc.sents:
        for token in sent:
            if token.pos_ in ['NOUN', 'ADJ', 'VERB','ADV']:#名詞、形容詞、動詞、副詞のみ
                result_list.append(token.text)
            
        #https://yu-nix.com/blog/2021/3/3/spacy-pos-list/#spaCy%E3%81%AE%E5%93%81%E8%A9%9E%E4%B8%80%E8%A6%A7%EF%BC%88%E6%97%A5%E6%9C%AC%E8%AA%9E%E8%A8%B3%EF%BC%89
        #result_list = result_list + [token.text for token in sent if token.pos_ in ['NOUN', 'ADJ', 'VERB','ADV','SCONJ','PART','DET','AUX']]

    return result_list

In [ ]:
#Mecabアナライザ
def mecab_analyzer(text):
    #正規化
    text = unicodedata.normalize("NFKC", text)
    #text = re.sub(r'\u3000','',text)
    text.replace('\u3000', '')
    tagger = MeCab.Tagger("-Owakati")
    node = tagger.parseToNode(text)

    # 指定した品詞を抽出しリストに
    word_list = []
    
    while node:
        word_type = node.feature.split(',')[0]
        if word_type in ['名詞', '形容詞', '動詞','副詞','接尾辞']:
            word_list.append(node.surface)
        #ここのインデントミスると無限ループ
        node = node.next
    # リストを文字列に変換
    word_chain = ' '.join(word_list)
    return word_list

In [ ]:
#全投稿のコサイン類似度を判定したい
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


#token格納用リスト
token_list = []

for review in tqdm(company_df.本文):
    review_token = mecab_analyzer(review)
    token_list.append(' '.join(review_token))

In [ ]:
#TfIdfインスタンス作成
vec = TfidfVectorizer(analyzer=mecab_analyzer,
                      #stop_words=stopword_list,
                      min_df=10,#あまりにも出てこないワードは対象外
                      ngram_range=(1,2),use_idf=True)#bigram/unigram/を考慮する

In [ ]:
#tfidf行列を作成
vec.fit(token_list)

In [ ]:
#tfidf行列作成
tfidf_matrix = vec.transform(token_list)

In [ ]:
#save
np.save('./array/tfidf_matrix.pkl', tfidf_matrix, allow_pickle=True)

In [ ]:
#load
np.load('./array/tfidf_matrix.pkl.npy', allow_pickle=True, encoding='ASCII')

In [ ]:
#w2v model load
import gensim
model = gensim.models.KeyedVectors.load('./model/w2v_nottrained_skip.model')

w2v_words = list(model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

In [ ]:
#model.wv.most_similar('優秀')
#model.wv['年功序列']

In [ ]:
#get vocab and tfidf value dict from tfidf instance
dictionary = dict(zip(vec.get_feature_names(), list(vec.idf_)))

In [ ]:
# TF-IDF weighted Word2Vec
tfidf_feat = vec.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;

for review in company_df.本文: # for each review/sentence 
    sent_vec = np.zeros(200) # as word vectors are of zero length
    weight_sum =0 # num of words with a valid vector in the sentence/review
    
    tagger = MeCab.Tagger("-Owakati")
    node = tagger.parseToNode(review)

    # 指定した品詞を抽出しリストに
    word_list = []
    
    while node:
        word_type = node.feature.split(',')[0]
        if word_type in ['名詞', '形容詞', '動詞','副詞','接尾辞']:
            word_list.append(node.surface)
        #ここのインデントミスると無限ループ
        node = node.next
    
    for word in word_list: # for each word in a review
        if word in w2v_words and word in tfidf_feat:
            vec = model.wv[word]#200次元のW2Vベクトル
            #tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            
            #tfidf行列の中の対応する単語のidf値に、レビューごとの
            #その単語の登場回数を、レビューの単語数で割ったもの(tf)をかけて、tfidf値としている
            tfidf = dictionary[word]*(review.count(word)/len(review))
            
            #w2v単語ベクトルに、tfidf値をかけることで、tfidfかけたw2v単語ベクトルを獲得
            sent_vec += (vec * tfidf)
            
            #各単語のtfidf値を合計している。
            #あるレビューにおける、各単語のtfidf値の合計は、word2vecの各単語にtfidfの重みをかけた
            #加重word2vecベクトルに対して、その重みの合計値で割ることで、加重平均ベクトルを
            #抽出するために必要なのである。
            weight_sum += tfidf
            
    #かけたものを、文章内の単語のtfidf値合計で割ることで、tfidf加重平均word2vecを出している
    if weight_sum != 0:
        sent_vec /= weight_sum
    #ここに重みつけしたベクトルが入る
    tfidf_sent_vectors.append(sent_vec)
    row += 1
    print(row) #進捗確認用

In [ ]:
#ちゃんとtfidfで加重平均とったw2vベクトルが獲得できた
#arrayにしておく
tfidf_sent_vectors_array = np.array(tfidf_sent_vectors)
tfidf_sent_vectors_array.shape

In [ ]:
#保存
np.save('./array/tfidf_matrix_fixed.pkl', tfidf_sent_vectors_array, allow_pickle=True)

In [ ]:
# 任意のテキストのベクトルを計算する関数
def get_vector(text):
    sum_vec = np.zeros(200)
    word_count = 0
    mecab = MeCab.Tagger('-Owakati')
    node = mecab.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        # 名詞、動詞、形容詞に限定
        if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞' or fields[0] == '副詞' or fields[0] == '接尾辞':
            sum_vec += model[node.surface]
            word_count += 1
        node = node.next
    return sum_vec / word_count#全単語ベクトルの平均をとっているだけ

In [ ]:
get_vector('年功序列')

In [ ]:
#完成版
def similar_company_f(input_text):
    text_similarity = cosine_similarity(get_vector(input_text).reshape(-1,200), tfidf_sent_vectors_array.reshape(-1,200))
    # 類似度が0.3以上の要素の数を取り出す
    num_of_similarities = (np.sum([x > 0.3 for x in text_similarity]))
    #類似度でsortとargsortして、類似度とインデックス双方を取り出す
    #topは表示件数
    top = 20
    top_indices = np.argsort(-text_similarity)[0][:num_of_similarities].tolist()[:top+1]
    top_similarity = np.sort(text_similarity.ravel())[::-1][:num_of_similarities].tolist()[:top+1]

    #dfのハコを作り
    answer_df = pd.DataFrame(columns=['類似度','企業名', '評価', '本文', '職種', '経験年数','現職/退職','新卒/中途','性別'])
    #forループで連結する
    for a,b in zip(top_indices,top_similarity):
        index_data = company_df.iloc[a]
        answer_df =answer_df.append({'類似度': b,'企業名': index_data[0], 
                                     '評価': index_data[4],'本文':index_data[5],
                                     '職種':index_data[8],'経験年数':index_data[9],
                                     '現職/退職':index_data[10],'新卒/中途':index_data[11],
                                     '性別':index_data[12]},ignore_index=True)
    
    #出来上がったdfを、company_info_dfとマージ
    answer_df_marged = pd.merge(answer_df,pd.DataFrame(company_info_df[['企業名','総合スコア']]),on='企業名',how='inner')
    
    #出来上がったdfを、類以度でsort
    answer_df_marged_g = answer_df_marged.set_index(['総合スコア','企業名'])
    #最後は総合スコアでSortしたものを返す
    #return answer_df_marged_g.sort_index(ascending=False)[:top].sort_index(ascending=False)[:top]
    return answer_df_marged_g.sort_values(by='類似度',ascending=False)[:top]

similar_company_f('残業が少なく働きやすい')

In [ ]:
# 単語のベクトルを平均を取るのではなく全部保存する関数
def get_vector_mod(text):
    blank = []
    sum_vec = np.zeros(200)
    #word_count = 0
    mecab = MeCab.Tagger('-Owakati')
    node = mecab.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        # 名詞、動詞、形容詞に限定
        if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
            blank.append(model[node.surface])
        node = node.next
        
    return np.array(blank)

In [ ]:
get_vector_mod('会社で働く').shape #形態素単位でベクトルが保存されている

In [ ]:
model.wv.most_similar('残業',topn=1000)
model.wv["残業"].shape

In [ ]:
def each_word_similarity(input_word):
    
    review = """平均的に年齢も若く、ワイワイノリの雰囲気があります。
    　　　　　　皆頑張って働いていて若手が引っ張っていくという考えが若手全員にありました。
          　　　上層部でもそのような考え方には否定的でなくむしろ肯定的で頑張っている若手には
             　 チャンスを多く与えてくれました。"""
    
    review = """若手が活躍している。"""
    
    similarity_list = []
    
    #inputを形態素に分ける

    tagger = MeCab.Tagger("-Owakati")
    node = tagger.parseToNode(input_word)

    # 指定した品詞を抽出しリストに
    input_word_list = []
    while node:
        word_type = node.feature.split(',')[0]
        if word_type in ['名詞', '形容詞', '副詞', '動詞']:
            input_word_list.append(node.surface)
        #ここのインデントミスると無限ループ
        node = node.next
        
    
    #レビューを形態素に分ける
    tagger2 = MeCab.Tagger("-Owakati")
    node2 = tagger.parseToNode(review)

    # 指定した品詞を抽出しリストに
    input_word_list2 = []
    
    while node2:
        word_type = node2.feature.split(',')[0]
        if word_type in ['名詞', '形容詞', '副詞', '動詞']:
            input_word_list2.append(node2.surface)
        #ここのインデントミスると無限ループ
        node2 = node2.next
    
    #単語単位での類似性判定
    for ei in input_word_list:
        for er in input_word_list2:
            if er in [al[0] for al in model.wv.most_similar(ei,topn=100)] + [ei]:
            try:
                similarity = cosine_similarity(model.wv[ei].reshape(-1,200), model.wv[er].reshape(-1,200)).ravel()[0]
                #similarity = cosine_similarity(get_vector_mod(ei).reshape(-1,200), model.wv[er].reshape(-1,200)).ravel()[0]
                similarity_list.append(similarity)
            except ValueError:
                pass
                
    print(similarity_list)
    print(input_word_list)
    print(input_word_list2)
    #単語ベースでの平均類以度を算出
    return np.mean(similarity_list)

In [ ]:
review = """若手が活躍している"""
each_word_similarity(review) 

In [ ]:
# 指定した品詞を抽出しリストに
tagger = MeCab.Tagger("-Owakati")
node = tagger.parseToNode('若手が活躍している')
input_word_list = []
while node:
    word_type = node.feature.split(',')[0]
    if word_type in ['名詞', '形容詞', '副詞', '動詞']:
        input_word_list.append(node.surface)
        #ここのインデントミスると無限ループ
    node = node.next
    
input_word_list

In [ ]:
# TF-IDF weighted Word2Vec
tfidf_feat = vec.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;

for review in company_df.本文: # for each review/sentence 
    sent_vec = np.zeros(200) # as word vectors are of zero length
    weight_sum =0 # num of words with a valid vector in the sentence/review
    
    tagger = MeCab.Tagger("-Owakati")
    node = tagger.parseToNode(review)

    # 指定した品詞を抽出しリストに
    word_list = []
    
    while node:
        word_type = node.feature.split(',')[0]
        if word_type in ['名詞', '形容詞', '副詞', '動詞']:
            word_list.append(node.surface)
        #ここのインデントミスると無限ループ
        node = node.next
    
    for word in word_list: # for each word in a review
        if word in w2v_words and word in tfidf_feat:
            vec = model.wv[word]#200次元のW2Vベクトル
            #tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            
            #tfidf行列の中の対応する単語のidf値に、レビューごとの
            #その単語の登場回数を、レビューの単語数で割ったもの(tf)をかけて、tfidf値としている
            tfidf = dictionary[word]*(review.count(word)/len(review))
            
            #w2v単語ベクトルに、tfidf値をかけることで、tfidfかけたw2v単語ベクトルを獲得
            sent_vec += (vec * tfidf)
            
            #各単語のtfidf値を合計している。
            #あるレビューにおける、各単語のtfidf値の合計は、word2vecの各単語にtfidfの重みをかけた
            #加重word2vecベクトルに対して、その重みの合計値で割ることで、加重平均ベクトルを
            #抽出するために必要なのである。
            weight_sum += tfidf
            
    #かけたものを、文章内の単語のtfidf値合計で割ることで、tfidf加重平均word2vecを出している
    if weight_sum != 0:
        sent_vec /= weight_sum
    #ここに重みつけしたベクトルが入る
    tfidf_sent_vectors.append(sent_vec)
    row += 1
    print(row) #進捗確認用

In [ ]:
#ベクトルの平均を取るのではなく、inputtextに類似する単語の類以度を平均する方法
def similar_company_f(input_text):
    text_similarity = cosine_similarity(get_vector(input_text).reshape(-1,200), tfidf_sent_vectors_array.reshape(-1,200))
    # 類似度が0.3以上の要素の数を取り出す
    num_of_similarities = (np.sum([x > 0.3 for x in text_similarity]))
    #類似度でsortとargsortして、類似度とインデックス双方を取り出す
    #topは表示件数
    top = 20
    top_indices = np.argsort(-text_similarity)[0][:num_of_similarities].tolist()[:top+1]
    top_similarity = np.sort(text_similarity.ravel())[::-1][:num_of_similarities].tolist()[:top+1]

    #dfのハコを作り
    answer_df = pd.DataFrame(columns=['類似度','企業名', '評価', '本文', '職種', '経験年数','現職/退職','新卒/中途','性別'])
    #forループで連結する
    for a,b in zip(top_indices,top_similarity):
        index_data = company_df.iloc[a]
        answer_df =answer_df.append({'類似度': b,'企業名': index_data[0], 
                                     '評価': index_data[4],'本文':index_data[5],
                                     '職種':index_data[8],'経験年数':index_data[9],
                                     '現職/退職':index_data[10],'新卒/中途':index_data[11],
                                     '性別':index_data[12]},ignore_index=True)
    
    #出来上がったdfを、company_info_dfとマージ
    answer_df_marged = pd.merge(answer_df,pd.DataFrame(company_info_df[['企業名','総合スコア']]),on='企業名',how='inner')
    
    #出来上がったdfを、類以度でsort
    answer_df_marged_g = answer_df_marged.set_index(['総合スコア','企業名'])
    #最後は総合スコアでSortしたものを返す
    #return answer_df_marged_g.sort_index(ascending=False)[:top].sort_index(ascending=False)[:top]
    return answer_df_marged_g.sort_values(by='類似度',ascending=False)[:top]

similar_company_f('雰囲気が良い')

In [ ]:
import pickle

tfidf_matrix = pickle.load(open("tfidf_matrix.pkl", "rb"))

vec = pickle.load(open("tfidf_vec.pkl", "rb"))
vec.set_params(analyzer=mecab_analyzer)

In [ ]:
import pickle
vec.set_params(analyzer='word')
# 保存 
pickle.dump(vec, open("tfidf_vec.pkl", "wb"))
pickle.dump(tfidf_matrix,open('tfidf_matrix.pkl',"wb"))

In [ ]:
jobtype_dict = {}
with open('./jobtype.csv','r',encoding='cp932') as f:
    for k in f:
        ab = k.strip().split(',')
        if ab[0] not in jobtype_dict:
            jobtype_dict[ab[0]] = ab[1]

            

#職種の分類は以下の通り        
"""
その他
コーポレート・管理・総務
営業職
事務
マーケティング職
製造
研究開発
人事
開発・設計
企画系
経理財務
SE/PM系
マネジメント
エンジニア系
デザイナー
法務

"""
#以下にjobtypeの辞書を用意した
jobtype_list_dict = {0:'その他',1:"コーポレート・管理・総務",2:'営業職・販売',3:'事務',4:'マーケティング職',
                5:'製造',6:'研究開発',7:'人事',8:'開発・設計・技術',9:'生産管理・調達・物流',10:'マネジメント',11:'企画系',
                12:'現業',13:'経理財務',14:'SE/PM系',15:'エンジニア系',16:'専門職',17:'コンサルタント',18:'デザイナー',19:'法務'}


In [ ]:
#任意の文字列の入力と、職種IDをうけとり、指定した職種IDにマッチし、入力と類似したレビューを持つ企業名とレビューを返す関数
def similar_company(input_text,jobtype=None):
    '''
    見たいレビューの職種を限定したい場合は
    以下のjobtype_listのindex番号を引数で渡し、指定すること(指定しなくてもOK)
    (基本的には「その他」は分類不能なもの(「総合職」、「本社」、「～～会社xx事業部」)が多いです)
    
jobtype_list_dict = {0:'その他',1:"コーポレート・管理・総務",2:'営業職・販売',3:'事務',4:'マーケティング職',
                5:'製造',6:'研究開発',7:'人事',8:'開発・設計・技術',9:'生産管理・調達・物流',10:'マネジメント',11:'企画系',
                12:'現業',13:'経理財務',14:'SE/PM系',15:'エンジニア系',16:'専門職',17:'コンサルタント',18:'デザイナー',19:'法務'}
    
    '''
    #tokenization
    tokens = mecab_analyzer(input_text)
    #トークンを格納したリストを作成
    token_list = []
    token_list.append(' '.join(tokens))
    sample_vector = vec.transform(token_list)

    # 計算した文書のtfidf_matrixと指定した文字列のベクトルのコサイン類似度を計算
    text_similarity = cosine_similarity(sample_vector, tfidf_matrix)
    
    # 類似度が0.4以上の要素の数を取り出す
    num_of_similarities = (np.sum([x > 0.3 for x in text_similarity]))

    #類似度でsortとargsortして、類似度とインデックス双方を取り出す
    top_indices = np.argsort(-text_similarity)[0][:num_of_similarities].tolist()
    top_similarity = np.sort(text_similarity.ravel())[::-1][:num_of_similarities].tolist()
    
    #dfのハコを作り
    answer_df = pd.DataFrame(columns=['類似度','企業名', '評価', '本文', '職種', '経験年数','現職/退職','新卒/中途','性別'])
    
    #カラム内の文字数。デフォルトは50なので変更
    pd.set_option("display.max_colwidth", 1500)
    
    #行数上限も変更し
    pd.set_option("display.max_rows", 101)

    #forループで連結する
    for a,b in zip(top_indices,top_similarity):
        index_data = company_df.iloc[a]
        answer_df =answer_df.append({'類似度': b,'企業名': index_data[0], 
                                     '評価': index_data[4],'本文':index_data[5],
                                     '職種':index_data[8],'経験年数':index_data[9],
                                     '現職/退職':index_data[10],'新卒/中途':index_data[11],
                                     '性別':index_data[12]},ignore_index=True)
        
    #出来上がったdfを、company_info_dfとマージ
    answer_df_marged = pd.merge(answer_df,pd.DataFrame(company_info_df[['企業名','総合スコア']]),on='企業名',how='inner')
    
    #企業風土の評価スコアはdrop
    #answer_df_marged = answer_df_marged.drop('評価',axis=1)
    #出来上がったdfを、類以度でsort　topは表示件数
    top = 30
    answer_df_marged_g = answer_df_marged.set_index(['総合スコア','企業名'])
    
    #jobtypeの入力に応じて出力を変える
    #会社の総合スコア順に表示
    
    if jobtype is None:
        return answer_df_marged_g.sort_index(ascending=False)[:top]
    
    else:
        answer_df_marged_g['職種'] =  answer_df_marged_g['職種'].map(jobtype_dict)
        job_adm = answer_df_marged_g[answer_df_marged_g['職種'] == jobtype_list_dict[jobtype]].reset_index()
        job_adm_s = job_adm.set_index(['総合スコア','企業名'])
        return job_adm_s.sort_index(ascending=False)[:top]
        
        
        
    #return answer_df_marged_g[:top].sort_values('総合スコア',ascending=False)

    #return answer_df_marged.sort_values(['総合スコア'],ascending=False).reset_index().drop('index',axis=1)[:top]


#ここに検索したい企業文化をフリーワードで入力する
input_text = '残業が少なくて働きやすい'
similar_company(input_text)

In [ ]:
import MeCab
from gensim.models import KeyedVectors
import numpy as np
from gensim.models import word2vec
import logging
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


In [ ]:
# モデルのロード(モデルが用意してあれば、ここからで良い)
m = Doc2Vec.load('./model/openwork.model')

In [ ]:
#企業ごとコメント全部をベクトル化
#まず企業毎のレビューを全部くっつけて、1企業全レビューのdfにする
all_docs = pd.DataFrame(company_df.groupby('企業名').本文.apply(list).apply(' '.join))

In [ ]:
#token格納用リスト
token_list = []
    
for review in all_docs.本文:
    review_token = mecab_analyzer(review)
    token_list.append(' '.join(review_token))
    
trainings = [TaggedDocument(words = data.split(' '),tags = [i]) for i,data in enumerate(token_list)]

#Doc2Vecをためす
#parameter: https://github.com/jkatsuta/18_4q_doc2vec/blob/master/experiments/exp_params_setup1.ipynb
model = Doc2Vec(documents=trainings, dm=1,
                min_alpha=1e-4, size=200,
                window=10, min_count=10,
                sample=1e-3,
                worker=-1,epochs=150#epoch=150
                )


print('\n訓練開始')
for epoch in range(150):
    print('Epoch: {}'.format(epoch + 1))
    model.train(trainings,total_examples=model.corpus_count, epochs=model.epochs)


# モデルのセーブ
model.save("./model/openwork_uniquecompany_dbow.model")


In [ ]:
compname = 'ヤフー株式会社'

In [ ]:
# モデルのロード(モデルが用意してあれば、ここからで良い)
#m = Doc2Vec.load("./model/openwork_uniquecompany_dbow.model")
m = Doc2Vec.load("./model/old/openwork_uniquecompany.model")

#企業名を引数にとり、その企業のコメントベクトルに類似したベクトルを持つ企業名と類似度を表示する関数
def company_comparison(company_name):
    identified_company_docs = all_docs[all_docs.index == company_name]
    input_text = identified_company_docs.本文.str.cat()
    #Mecabによって形態素解析
    mecab_input = mecab_analyzer(input_text)
    
    #docvecsによる類似性判定結果
    #類似度TOP10の企業を表示
    similarity = (m.docvecs.most_similar(positive=[m.infer_vector(mecab_input)],topn=10))
    
    #格納用空リスト
    return_list=[]
    for num,similarity in similarity:
        return_list.append([all_docs.index[num],similarity])
    #df化
    output_df = pd.DataFrame(return_list,columns=['企業名','類似度']).sort_values('類似度',ascending=False)
    return output_df[1:] #自社が最も類似しているので、1行目は除いて、2行目から表示
    
    
company_comparison(compname)

In [ ]:
m = Doc2Vec.load("./model/openwork_uniquecompany_dbow.model")
#m = Doc2Vec.load("./model/old/openwork_uniquecompany.model")

#企業名を引数にとり、その企業のコメントベクトルに類似したベクトルを持つ企業名と類似度を表示する関数
def company_comparison(company_name):
    identified_company_docs = all_docs[all_docs.index == company_name]
    input_text = identified_company_docs.本文.str.cat()
    #Mecabによって形態素解析
    mecab_input = mecab_analyzer(input_text)
    
    #docvecsによる類似性判定結果
    #類似度TOP10の企業を表示
    similarity = (m.docvecs.most_similar(positive=[m.infer_vector(mecab_input)],topn=10))
    
    #格納用空リスト
    return_list=[]
    for num,similarity in similarity:
        return_list.append([all_docs.index[num],similarity])
    #df化
    output_df = pd.DataFrame(return_list,columns=['企業名','類似度']).sort_values('類似度',ascending=False)
    return output_df[1:] #自社が最も類似しているので、1行目は除いて、2行目から表示
    
    
company_comparison(compname)

In [ ]:
#検索したいワード
input_text = '残業が少なく働きやすい'

#Mecabによって形態素解析
mecab_input = mecab_analyzer(input_text)

#docvecsプロパティのmost_similarメソッドを使用すると、
#ラベル付した記事からコサイン類似度を計算して類似した記事を取得することができる
#類似文書のindexを取得
#positive:類似 negative:類似していない
print(m.docvecs.most_similar(positive=[m.infer_vector(mecab_input)],topn=5))

In [ ]:
#企業毎にコメント数がバラバラな問題があるので、抽出件数を揃えたい
#企業ごとコメントを8件RANDOMに抽出し、ベクトル化したらどうなるか
AK = pd.DataFrame(company_df.groupby('企業名').本文.apply(lambda x:x.sample(n=5,random_state=0,replace=True)))
#df.sampleにかんして参照 https://analytics-note.xyz/programming/dataframe-sample-doc/

all_docs_5sentences = pd.DataFrame(AK.groupby('企業名').本文.apply(list).apply(''.join))

#token格納用リスト
token_list = []
    
for review in all_docs_5sentences.本文:
    review_token = mecab_analyzer(review)
    token_list.append(' '.join(review_token))
    
trainings = [TaggedDocument(words = data.split(' '),tags = [i]) for i,data in enumerate(token_list)]

#parameter: https://github.com/jkatsuta/18_4q_doc2vec/blob/master/experiments/exp_params_setup1.ipynb
model = Doc2Vec(documents=trainings, dm=1,
                min_alpha=.1e-4, size=200,
                window=10, min_count=5,
                worker=-1,epochs=50
                )


print('\n訓練開始')
for epoch in range(50):
    print('Epoch: {}'.format(epoch + 1))
    model.train(trainings,total_examples=model.corpus_count, epochs=model.epochs)


# モデルのセーブ
model.save("./model/openwork_uniquecompany_8sentences.model")


In [ ]:
# モデルのロード(モデルが用意してあれば、ここからで良い)
m = Doc2Vec.load("./model/openwork_uniquecompany_8sentences.model")

#企業名を引数にとり、その企業のコメントベクトルに類似したベクトルを持つ企業名と類似度を表示する関数
def company_comparison(company_name):
    identified_company_docs = all_docs[all_docs.index == company_name]
    input_text = identified_company_docs.本文.str.cat()
    #Mecabによって形態素解析
    mecab_input = mecab_analyzer(input_text)
    
    #docvecsによる類似性判定結果
    #類似度TOP10の企業を表示
    similarity = (m.docvecs.most_similar(positive=[m.infer_vector(mecab_input)],topn=10))
    
    #格納用空リスト
    return_list=[]
    for num,similarity in similarity:
        return_list.append([all_docs.index[num],similarity])
    #df化
    output_df = pd.DataFrame(return_list,columns=['企業名','類似度']).sort_values('類似度',ascending=False)
    return output_df[1:] #自社が最も類似しているので、1行目は除いて、2行目から表示
    
company_comparison(compname)

In [ ]:
# 数値カラムを用意
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaling_columns = ['月残業時間', '有給消化率', '平均年収', '社員の士気', '風通し', '社員相互尊重',
       '20代成長性', '長期育成', 'コンプラ',  '待遇満足度', '会社年齢'] 

#StandardScale 使うのはこっちのデータ
SS = StandardScaler()
SS.fit(company_info_df_processed[scaling_columns])

Sdata = pd.DataFrame(SS.transform(company_info_df_processed[scaling_columns]),columns=scaling_columns)

Sdata_m = pd.concat([Sdata,pd.DataFrame(company_info_df_processed.企業名).reset_index()], axis=1,
                     ignore_index=True)

Sdata_m = Sdata_m.drop(11,axis=1)

Sdata_m.columns=['月残業時間', '有給消化率', '平均年収', '社員の士気', '風通し', '社員相互尊重',
       '20代成長性', '長期育成', 'コンプラ',  '待遇満足度', '会社年齢','企業名']

Sdata_m

In [ ]:
#doc2vecの類以度(top1類以度)の、company_info_dfベクトルでみたときの類似度との二乗誤差を計算
def cos_sim_error(company_name):
    m = Doc2Vec.load("./model/openwork_uniquecompany_dbow.model")
    #m = Doc2Vec.load("./model/old/openwork_uniquecompany.model")
    identified_company_docs = all_docs[all_docs.index == company_name]
    input_text = identified_company_docs.本文.str.cat()
    #Mecabによって形態素解析
    mecab_input = mecab_analyzer(input_text)
    
    #doc2vecによる類似性判定結果
    #類似度TOP2の企業を表示
    similarity = (m.docvecs.most_similar(positive=[m.infer_vector(mecab_input)],topn=2))
    
    #自社は一番目なので2番めのindex(compname)を取り出す
    most_similar_compname =  all_docs.index[similarity[1][0]]
    #その類以度も取り出す
    doc2vec_similarity =  similarity[1][1]
    
    #Doc2Vecが指示した、最も類似した企業名を使って、company_info_dfのベクトルでの類似度を求める
    #例外は用意しておく(doc2vecが提示した類似企業がcompany_info_dfにない場合があるので)
    try:
        cinfo_sim = cosine_similarity(Sdata_m[Sdata_m.企業名 == company_name].drop('企業名',axis=1), Sdata_m[Sdata_m.企業名 == most_similar_compname].drop('企業名',axis=1))
        #それぞれの類以度の二乗誤差を返す
        se_cinfo_sim = ((cinfo_sim - doc2vec_similarity)**2).ravel()[0]
        
    except ValueError:
        #doc2vecが提示した類似企業がcompany_info_dfにない場合,二乗誤差はNANとする
        se_cinfo_sim = np.nan
    
    finally:
        return se_cinfo_sim

#レビューを持つ企業全てに対して、doc2vecでの類以度と、company_info_dfベクトルの類似度で、top1類以度の企業の類以度二乗誤差を計算
se = Sdata_m.企業名.apply(cos_sim_error)
se

In [ ]:
#doc2vecの類以度(top1類以度)の、company_info_dfベクトルでみたときの類似度との二乗誤差を計算
def cos_sim_error_50(company_name):
    #m = Doc2Vec.load("./model/openwork_uniquecompany_dbow.model")
    m = Doc2Vec.load("./model/old/openwork_uniquecompany.model")
    identified_company_docs = all_docs[all_docs.index == company_name]
    input_text = identified_company_docs.本文.str.cat()
    #Mecabによって形態素解析
    mecab_input = mecab_analyzer(input_text)
    
    #doc2vecによる類似性判定結果
    #類似度TOP2の企業を表示
    similarity = (m.docvecs.most_similar(positive=[m.infer_vector(mecab_input)],topn=2))
    
    #自社は一番目なので2番めのindex(compname)を取り出す
    most_similar_compname =  all_docs.index[similarity[1][0]]
    #その類以度も取り出す
    doc2vec_similarity =  similarity[1][1]
    
    #Doc2Vecが指示した、最も類似した企業名を使って、company_info_dfのベクトルでの類似度を求める
    #例外は用意しておく(doc2vecが提示した類似企業がcompany_info_dfにない場合があるので)
    try:
        cinfo_sim = cosine_similarity(Sdata_m[Sdata_m.企業名 == company_name].drop('企業名',axis=1), Sdata_m[Sdata_m.企業名 == most_similar_compname].drop('企業名',axis=1))
        #それぞれの類以度の二乗誤差を返す
        se_cinfo_sim = ((cinfo_sim - doc2vec_similarity)**2).ravel()[0]
        
    except ValueError:
        #doc2vecが提示した類似企業がcompany_info_dfにない場合,二乗誤差はNANとする
        se_cinfo_sim = np.nan
    
    finally:
        return se_cinfo_sim

#レビューを持つ企業全てに対して、doc2vecでの類以度と、company_info_dfベクトルの類似度で、top1類以度の企業の類以度二乗誤差を計算
se50 = Sdata_m.企業名.apply(cos_sim_error_50)
se50

In [ ]:
#BM25を試す
import spacy
from rank_bm25 import BM25Okapi
from tqdm import tqdm
nlp = spacy.load('ja_ginza')
text_list = token_list
tok_text=[] # for our tokenised corpus

"""
token.text, # テキスト
token.lemma_, # レンマ
token.pos_, # 品詞
token.tag_, # 品詞詳細
token.dep_, # 構文従属関係
token.shape_, # 正書法の特徴(x:文字,d:数値)
token.is_alpha, # 文字かどうか
token.is_stop # ストップリストの一部かどうか
"""
#refrence: https://spacy.io/usage/processing-pipelines

#Tokenising using SpaCy:
#for doc in tqdm(nlp.pipe(text_list, disable=["tagger", "parser","ner"])):
#    tok = [t.text for t in doc if t.is_alpha]
#    tok_text.append(tok)

In [ ]:
#with open('./bygram_list/token_text.pkl', 'wb') as f:
#    pickle.dump(tok_text, f)

In [ ]:
from rank_bm25 import BM25Plus as BM25
bm25 = BM25(tok_text)

In [ ]:
query = "優秀な人が多い。スピード感がある。"
tokenized_query = mecab_analyzer(query)
import time
t0 = time.time()

#top n
n=10

results = bm25.get_top_n(tokenized_query, company_df.本文.values, n=n)
scores = bm25.get_scores(tokenized_query)
topnscores = np.sort(scores)[::-1][:n].tolist()

t1 = time.time()
print(f'Searched 39,285 records in {round(t1-t0,3) } seconds \n')
for i,k in zip(topnscores,results):
    print('{:.2f}'.format(i)+' '+k+'\n')

    

In [ ]:
scores = bm25.get_scores(tokenized_query)
np.min(scores)

In [ ]:
query = "優秀な人が多い。スピード感がある。"
tokenized_query = mecab_analyzer(query)
scores = bm25.get_scores(tokenized_query)
np.min(scores)

In [ ]:
#w2v array load
allsp_array = np.load('./array/w2v.pkl.npy',allow_pickle=True)
allsp_array_fixed = np.array(pd.DataFrame(allsp_array).fillna(0))

#query to vector
text_vector = get_vector(query)

#文書間のcos類似度を計算 
text_similarity = cosine_similarity(text_vector.reshape(-1,200), allsp_array_fixed).flatten()

#BM25のスコアと,W2Vでの類以度同士の掛け算を行う
BM25W2V = np.array(text_similarity * np.array(scores))

print(BM25W2V)


In [ ]:
#類似度でsortとargsortして、類似度とインデックス双方を取り出す topn件
top=20
top_indices = np.argsort(-BM25W2V).tolist()[:top+1]
top_similarity = np.sort(BM25W2V)[::-1].tolist()[:top+1]

#dfのハコを作り
answer_df = pd.DataFrame(columns=['類似度','企業名', '評価', '本文', '職種', '経験年数','現職/退職','新卒/中途','性別'])
    
#カラム内の文字数。デフォルトは50なので変更
pd.set_option("display.max_colwidth", 1500)
    
#行数上限も変更し
pd.set_option("display.max_rows", 101)

#forループで連結する
for a,b in zip(top_indices,top_similarity):
    index_data = company_df.iloc[a]
    answer_df =answer_df.append({'類似度': b,'企業名': index_data[0], 
                                     '評価': index_data[4],'本文':index_data[5],
                                     '職種':index_data[8],'経験年数':index_data[9],
                                     '現職/退職':index_data[10],'新卒/中途':index_data[11],
                                     '性別':index_data[12]},ignore_index=True)
        
    
#出来上がったdfを、company_info_dfとマージ
answer_df_marged = pd.merge(answer_df,pd.DataFrame(company_info_df[['企業名','総合スコア']]),on='企業名',how='inner')
    
#企業風土の評価スコアはdrop
#answer_df_marged = answer_df_marged.drop('評価',axis=1)
#出来上がったdfを、類以度でsort　topは表示件数

answer_df_marged_g = answer_df_marged.set_index(['総合スコア','企業名'])

answer_df_marged[:top]

In [ ]:
#bygramの辞書を作成するパート
#2gram類以度判定
bygram_list = []

#n_gram抽出関数
def n_gram(target, n):
  # 基準を1文字(単語)ずつ ずらしながらn文字分抜き出す
  return [target[idx:idx+n] for idx in range(len(target) - n + 1)]

for k in tqdm(token_list):
    words = k.split(' ')
    bygram_list.append(n_gram(words,2))
    
bygram_list

In [ ]:
query = '優秀な人が多い。スピード感がある。'

"""
#bygramのリストを作成するパート
#2gram類以度判定
bygram_list = []

#n_gram抽出関数
def n_gram(target, n):
  # 基準を1文字(単語)ずつ ずらしながらn文字分抜き出す
  return [target[idx:idx+n] for idx in range(len(target) - n + 1)]

for k in (tqdm(token_list)):
    words = k.split(' ')
    bygram_list.append(n_gram(words,2))
"""

In [ ]:
#with open('./bygram_list/bygram.pkl', 'wb') as f:
#    pickle.dump(bygram_list, f)

In [ ]:
with open('./bygram_list/bygram.pkl','rb') as f:
    bygram_list = pickle.load(f)

In [ ]:
bygram_score = []
tokenized_query = mecab_analyzer(query)

for p in tqdm(bygram_list):
    bygram_inclusion = 0
    
    for k in n_gram(tokenized_query,2):
        bygram_inclusion += p.count(k)
        
    #print(bygram_inclusion)
    
    try:
        #divider = len(p)+1/2
        bygram_score.append(bygram_inclusion/(len(p)-1))        
    except ZeroDivisionError:
        bygram_score.append(0)
        
print(bygram_score)

In [ ]:
np.argmax(bygram_score)
company_df.本文.iloc[9604]

In [ ]:
#w2v array load
allsp_array = np.load('./array/w2v.pkl.npy',allow_pickle=True)
allsp_array_fixed = np.array(pd.DataFrame(allsp_array).fillna(0))

#query to vector
text_vector = get_vector(query)
#文書間のcos類似度を計算 
text_similarity = cosine_similarity(text_vector.reshape(-1,200), allsp_array_fixed).flatten()

#W2Vでの類以度+　BM25スコア*Bygram単位での類以度
#bygramw2v = np.array(text_similarity + (np.array(scores) * np.array(bygram_score)))
bygramw2v = np.array(bygram_score)

#類似度でsortとargsortして、類似度とインデックス双方を取り出す topn件
top=20
top_indices = np.argsort(-bygramw2v).tolist()[:top+1]
top_similarity = np.sort(bygramw2v)[::-1].tolist()[:top+1]

#dfのハコを作り
answer_df = pd.DataFrame(columns=['類似度','企業名', '評価', '本文', '職種', '経験年数','現職/退職','新卒/中途','性別'])
    
#カラム内の文字数。デフォルトは50なので変更
pd.set_option("display.max_colwidth", 1500)
    
#行数上限も変更し
pd.set_option("display.max_rows", 101)

#forループで連結する
for a,b in zip(top_indices,top_similarity):
    index_data = company_df.iloc[a]
    answer_df =answer_df.append({'類似度': b,'企業名': index_data[0], 
                                     '評価': index_data[4],'本文':index_data[5],
                                     '職種':index_data[8],'経験年数':index_data[9],
                                     '現職/退職':index_data[10],'新卒/中途':index_data[11],
                                     '性別':index_data[12]},ignore_index=True)


#出来上がったdfを、company_info_dfとマージ
answer_df_marged = pd.merge(answer_df,company_info_df[['企業名','総合スコア']],on='企業名',how='left')

#企業風土の評価スコアはdrop
#answer_df_marged = answer_df_marged.drop('評価',axis=1)
#出来上がったdfを、類以度でsort　topは表示件数

answer_df_marged_g = answer_df_marged.set_index(['総合スコア','企業名'])
answer_df_marged.sort_values(by='類似度',ascending=False)[:top]

In [ ]:
#最終完成形検索アルゴリズム
from rank_bm25 import BM25Plus
import spacy
import gensim
import pickle
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

#トークンリスト読み込み
with open('./bygram_list/token_text.pkl','rb') as f:
    tok_text = pickle.load(f)

#w2vで作成した単語array読み込み
allsp_array = np.load('./array/w2v.pkl.npy',allow_pickle=True)
allsp_array_fixed = np.array(pd.DataFrame(allsp_array).fillna(0))

#レビュー本文のbygramリスト読み込み
with open('./bygram_list/bygram.pkl','rb') as f:
    bygram_list = pickle.load(f)

#tfidfでおもみ付けしたw2vベクトル
tfidf_sent_vectors_array = np.load('./array/tfidf_matrix_fixed.pkl.npy',allow_pickle=True)

#W2V model load
model = gensim.models.KeyedVectors.load('./model/w2v_nottrained_skip.model')

#n-gram func
def n_gram(target, n):
  # 基準を1文字(単語)ずつ ずらしながらn文字分抜き出す
  return [target[idx:idx+n] for idx in range(len(target) - n + 1)]

In [ ]:
def similar_company_f(input_text):
    #bm25plus
    bm25 = BM25Plus(tok_text)
    #クエリをトークン化
    tokenized_query = mecab_analyzer(input_text)

    #query to vector(w2vで使う)
    text_vector = get_vector(input_text)
    #bm25による全文のスコア算出(0-1のスケールになるように正規化)
    bm25numerator = np.array((bm25.get_scores(tokenized_query)) - np.min(np.array(bm25.get_scores(tokenized_query))))
    bm25denominator = (np.max(np.array(bm25.get_scores(tokenized_query))-np.min(np.array(bm25.get_scores(tokenized_query)))))
    BM25_mm_scores = bm25numerator/bm25denominator
    
    
    #文書間のcos類似度計算(検索クエリをW2V化 vs W2V行列)
    text_similarity = cosine_similarity(text_vector.reshape(-1,200), tfidf_sent_vectors_array.reshape(-1,200)).flatten()
    
    #各レビューとinputtextの、bygramでの一致スコアを計測する
    bygram_score = []
    
    for p in tqdm(bygram_list):
        bygram_inclusion = 0
        
        for k in n_gram(tokenized_query,2):
            bygram_inclusion += p.count(k)

    
        try:
            bygram_score.append(bygram_inclusion/len(p))        
        except ZeroDivisionError:
            bygram_score.append(0)
            
    #bygram_scoreも正規化しておく必要がある
    text_similarity_numerator = text_similarity - np.min(text_similarity)
    text_similarity_denominator = np.max(text_similarity) - np.min(text_similarity)
    text_similarity_mm_scores = text_similarity_numerator/text_similarity_denominator
    
    #text_similarityも
    bygramnumerator = np.array(bygram_score) - np.min(bygram_score)
    bygramdenominator = np.max(np.array(bygram_score)) - np.min(np.array(bygram_score))
    bygram_mm_scores = bygramnumerator/bygramdenominator
    
    
    #W2Vでの類以度 + BM25+のレビュー単位でのスコア + Bygram単位での類以度
    #足し算して平均しているだけ
    #bygramw2v = ((text_similarity_mm_scores) + (BM25_mm_scores) + bygram_mm_scores)/3
    #bygramw2v = (text_similarity + BM25_mm_scores + np.array(bygram_score))/3
    #bygramw2v = (BM25_mm_scores + np.array(bygram_score))
    #bygramw2v = (text_similarity) + np.array((bm25.get_scores(tokenized_query))) + (bygram_score)
    #bygramw2v = ((text_similarity_mm_scores) * (BM25_mm_scores) * (bygram_mm_scores))
    #bygramw2v = (np.array(bm25.get_scores(tokenized_query)) * np.array(bygram_score))
    bygramw2v = (bygram_mm_scores + 2*BM25_mm_scores)/3
    #bygramw2v = (BM25_mm_scores + bygram_mm_scores)/2 * text_similarity_mm_scores
    #類似度でsortとargsortして、類似度とインデックス双方を取り出す topn件
    top=20
    top_indices = np.argsort(-bygramw2v).tolist()[0:top+1]
    top_similarity = np.sort(bygramw2v)[::-1].tolist()[0:top+1]

    #dfのハコを作り
    answer_df = pd.DataFrame(columns=['類似度','企業名', '評価', '本文', '職種', '経験年数','現職/退職','新卒/中途','性別'])
    
    #カラム内の文字数。デフォルトは50なので変更
    pd.set_option("display.max_colwidth", 1500)
    
    #行数上限も変更し
    pd.set_option("display.max_rows", 101)

    #forループで連結する
    for a,b in zip(top_indices,top_similarity):
        index_data = company_df.iloc[a]
        answer_df =answer_df.append({'類似度': b,'企業名': index_data[0], 
                                     '評価': index_data[4],'本文':index_data[5],
                                     '職種':index_data[8],'経験年数':index_data[9],
                                     '現職/退職':index_data[10],'新卒/中途':index_data[11],
                                     '性別':index_data[12]},ignore_index=True)

    
    #出来上がったdfを、company_info_dfとマージ
    answer_df_marged = pd.merge(answer_df,pd.DataFrame(company_info_df[['企業名','総合スコア']]),on='企業名',how='left')
    

    #出来上がったdfを、類以度でsort
    answer_df_marged_g = answer_df_marged.set_index(['総合スコア','企業名'])

    return answer_df_marged.sort_values(by='類似度',ascending=False)

#similar_company_f('若手が活躍している')
#similar_company_f('残業が少ない')

#similar_company_f('残業が少なく働きやすい')
similar_company_f('実力主義で殺伐としている')

In [ ]:
n_gram(mecab_analyzer("残業が少なく働きやすい"), 2)

In [ ]:
mecab_analyzer('残業が少なく働きやすい')

In [ ]:
mecab_analyzer('実力主義の会社')

In [ ]:
import ginza
nlp = spacy.load('ja_ginza') # spacyにginzaを使用することを指定

txt = 'ゆるい雰囲気で人間関係も良好' # 入力文字列

doc = nlp(txt) # モデルへ適応
ginza.set_split_mode(nlp, "C") # 形態素の分割モード指定
result_list = []
for sent in doc.sents:
    result_list = result_list + [[str(token.i), token.text, token.lemma_, token.pos_, token.tag_] for token in sent]
    
result_list